## Using Vertex AI Vector Search and Vertex AI Embeddings for Text for StackOverflow Questions

### Overview

This lab demonstrates how to provide semantic search to a large dataset of questions from StackOverflow. Because of the large size of the dataset, you will query the questions using BigQuery. Then, you will create text embeddings from the questions using the [Vertex AI Text-Embeddings API](https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings) service and store them in a Cloud Storage bucket. Once all embeddings are created, you will create a [Vertex AI Vector Search](https://cloud.google.com/vertex-ai/docs/matching-engine/overview) index for those embeddings, so that you can search them afterwards.

The Vertex AI Text-Embeddings API enhances the process of generating text embeddings. These text embeddings, which are numerical representations of text, play a pivotal role in many tasks involving the identification of similar items, like Google searches, online shopping recommendations, and personalized music suggestions.

Vertex AI Vector Search Engine service is a high-scale, low-latency solution, for finding similar vectors from a large corpus. Vector Search is a fully managed offering, further reducing operational overhead. It is built upon [Approximate Nearest Neighbor (ANN) technology](https://ai.googleblog.com/2020/07/announcing-scann-efficient-vector.html) developed by Google Research.

Check out [this demo page](https://ai-demos.dev/demos/matching-engine) to see what you will accomplish at the end of this lab.

#### What you will learn:

- Query a public dataset using BigQuery.
- Convert a BigQuery dataset to embeddings using the Vertex AI Text-Embeddings API.
- Create an index in Vertex AI Vector Search.
- Upload embeddings to the index.
- Create an index endpoint.
- Deploy the index to the index endpoint.
- Perform an online query.

### Task 3. Set up the Jupyter notebook environment

In [1]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                        google-cloud-storage \
                        'google-cloud-bigquery[pandas]'

In [2]:
## Restart kernel
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
## Setup the environment values for your project
PROJECT = !gcloud config get-value project
PROJECT_ID = PROJECT[0]
REGION = "us-east4"

In [2]:
## Import and initialize the Vertex AI Python SDK.
import vertexai
vertexai.init(project = PROJECT_ID,
              location = REGION)

### Task 4. Prepare the data in BigQuery

The dataset used for this lab is the [StackOverflow dataset](https://console.cloud.google.com/marketplace/product/stack-exchange/stack-overflow). This public dataset is hosted in Google BigQuery and is included in BigQuery's 1TB/mo of free tier processing. This means that each user receives 1TB of free BigQuery processing every month, which can be used to run queries on this public dataset.

Stack Overflow is the largest online community for programmers to learn, share their knowledge, and advance their careers. Updated on a quarterly basis, this BigQuery dataset includes an archive of Stack Overflow content, including posts, votes, tags, and badges. This dataset is updated to mirror the Stack Overflow content on the Internet Archive, and is also available through the Stack Exchange Data Explorer.

The BigQuery table is too large to fit into memory, so you need to write a generator called `query_bigquery_chunks` to yield chunks of the dataframe for processing. Additionally, an extra column `title_with_body` is added, which is a concatenation of the question title and body that will be used for creating embeddings.

In [3]:
import math
from typing import Any, Generator

import pandas as pd
from google.cloud import bigquery

client = bigquery.Client(project=PROJECT_ID)

In [4]:
QUERY_TEMPLATE = """
        SELECT distinct q.id, q.title, q.body
        FROM (SELECT * FROM `bigquery-public-data.stackoverflow.posts_questions` where Score>0 ORDER BY View_Count desc) AS q
        LIMIT {limit} OFFSET {offset};
        """

In [5]:
def query_bigquery_chunks(
    max_rows: int, rows_per_chunk: int, start_chunk: int = 0
) -> Generator[pd.DataFrame, Any, None]:
    for offset in range(start_chunk, max_rows, rows_per_chunk):
        query = QUERY_TEMPLATE.format(limit=rows_per_chunk, offset=offset)
        query_job = client.query(query)
        rows = query_job.result()
        df = rows.to_dataframe()
        df["title_with_body"] = df.title + "\n" + df.body
        yield df

In [6]:
df = next(query_bigquery_chunks(max_rows=1000, rows_per_chunk=1000))

# Examine the data
df.head()

,id,title,body,title_with_body
0,33038954,DELETE Request with Body (Without using Deprec...,<p>I wanted to execute a HTTP DELETE request w...,DELETE Request with Body (Without using Deprec...
1,33018921,The difference between use print and return in...,<p>Okay what I want is remove the first elemen...,The difference between use print and return in...
2,31013637,How to create and implement a custom JavaScrip...,<p>I am new to javaScript and am unsure how to...,How to create and implement a custom JavaScrip...
3,31170323,Why isn't my variable preserved in drawRect?,<p>Im trying to have an nsview that takes a cu...,Why isn't my variable preserved in drawRect?\n...
4,31081131,Makefile always rebuilding,<p>I am using the following simple Makefile to...,Makefile always rebuilding\n<p>I am using the ...


### Task 5. Create text embeddings from BigQuery data

In [7]:
## Load the Vertex AI Embeddings for Text model.
from typing import List, Optional
from vertexai.preview.language_models import TextEmbeddingModel

model = TextEmbeddingModel.from_pretrained("text-embedding-004")

In [8]:
## Define an embedding method that uses the model.
def encode_texts_to_embeddings(sentences: List[str]) -> List[Optional[List[float]]]:
    try:
        embeddings = model.get_embeddings(sentences)
        return [embedding.values for embedding in embeddings]
    except Exception:
        return [None for _ in range(len(sentences))]

According to the documentation, each request can handle up to 5 text instances. So we will need to split the BigQuery question results in batches of 5 before sending to the embedding API.

In [9]:
## Create a generate_batches to split results in batches of 5 to be sent to the embeddings API.
import functools
import time
from concurrent.futures import ThreadPoolExecutor
from typing import Generator, List, Tuple

import numpy as np
from tqdm.auto import tqdm


# Generator function to yield batches of sentences
def generate_batches(
    sentences: List[str], batch_size: int
) -> Generator[List[str], None, None]:
    for i in range(0, len(sentences), batch_size):
        yield sentences[i : i + batch_size]

Encapsulate the process of generating batches and calling the embeddings API in a method called `encode_text_to_embedding_batched`. This method also handles rate-limiting using `time.sleep`. For production use cases, you would want a more sophisticated rate-limiting mechanism that takes retries into account.

In [10]:
def encode_text_to_embedding_batched(
    sentences: List[str], api_calls_per_second: int = 10, batch_size: int = 5
) -> Tuple[List[bool], np.ndarray]:

    embeddings_list: List[List[float]] = []

    # Prepare the batches using a generator
    batches = generate_batches(sentences, batch_size)

    seconds_per_job = 1 / api_calls_per_second

    with ThreadPoolExecutor() as executor:
        futures = []
        for batch in tqdm(
            batches, total=math.ceil(len(sentences) / batch_size), position=0
        ):
            futures.append(
                executor.submit(functools.partial(encode_texts_to_embeddings), batch)
            )
            time.sleep(seconds_per_job)

        for future in futures:
            embeddings_list.extend(future.result())

    is_successful = [
        embedding is not None for sentence, embedding in zip(sentences, embeddings_list)
    ]
    embeddings_list_successful = np.squeeze(
        np.stack([embedding for embedding in embeddings_list if embedding is not None])
    )
    return is_successful, embeddings_list_successful

In [11]:
## Test the encoding function by encoding a subset of data and see if the embeddings and distance metrics make sense.
# Encode a subset of questions for validation
questions = df.title.tolist()[:500]
is_successful, question_embeddings = encode_text_to_embedding_batched(
    sentences=df.title.tolist()[:500]
)

# Filter for successfully embedded sentences
questions = np.array(questions)[is_successful]

  0%|          | 0/100 [00:00<?, ?it/s]

In [12]:
## Save the dimension size for later usage when creating the Vertex AI Vector Search index.
DIMENSIONS = len(question_embeddings[0])

print(DIMENSIONS)

768


Sort questions in order of similarity. According to the [embedding documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings#colab_example_of_semantic_search_using_embeddings), the similarity of embeddings is calculated using the dot-product, with np.dot. Once you have the similarity score, sort the results and print them for inspection. 1 means very similar, 0 means very different.

In [13]:
import random

question_index = random.randint(0, 99)

print(f"Query question = {questions[question_index]}")

# Get similarity scores for each embedding by using dot-product.
scores = np.dot(question_embeddings[question_index], question_embeddings.T)

# Print top 20 matches
for index, (question, score) in enumerate(
    sorted(zip(questions, scores), key=lambda x: x[1], reverse=True)[:20]
):
    print(f"\t{index}: {question}: {score}")

Query question = google-api-translate-java Error retrieving translation
	0: google-api-translate-java Error retrieving translation: 0.9999982399825948
	1: App Engine import ssl Failed: 0.5704977581664283
	2: How can i know java library path?: 0.47841476591495913
	3: trying to get string into intent: 0.4653579043041147
	4: "unit tests have failed" for beautifulsoup: 0.46159190242648657
	5: link with correct href still gives 404: 0.44487738234949553
	6: ListView - ListView don't update: 0.42998236989400856
	7: Cordova/PhoneGap FileTransfer Uploaded File Doesn't Appear on Server: 0.42987655399784763
	8: stumped on clicking a link with nokogiri and mechanize: 0.4285047032693252
	9: std::unique_ptr<T> incomplete type error: 0.42488479310674543
	10: maven do not repack dependencies: 0.42009073961062865
	11: How to import a page break from html to google docs?: 0.4155192179397411
	12: Android View animation - poor performance on big screens: 0.4122582912556433
	13: TeamCity build runner not r

Save the embeddings in JSONL format. [The data must be formatted in JSONL format](https://cloud.google.com/vertex-ai/docs/matching-engine/match-eng-setup/format-structure#data-file-formats), which means each embedding dictionary is written as an individual JSON object on its own line.

In [14]:
import tempfile
from pathlib import Path

# Create temporary file to write embeddings to
embeddings_file_path = Path(tempfile.mkdtemp())

print(f"Embeddings directory: {embeddings_file_path}")

Embeddings directory: /var/tmp/tmp64_knyge


Write embeddings in batches to prevent out-of-memory errors. Notice we are only using 5000 questions so that the embedding creation process and indexing is faster. The dataset contains more than 50,000 questions. This step will take around 5 minutes.

In [16]:
import gc
import json

BQ_NUM_ROWS = 5000
BQ_CHUNK_SIZE = 1000
BQ_NUM_CHUNKS = math.ceil(BQ_NUM_ROWS / BQ_CHUNK_SIZE)

START_CHUNK = 0

# Create a rate limit of 300 requests per minute. Adjust this depending on your quota.
API_CALLS_PER_SECOND = 300 / 60
# According to the docs, each request can process 5 instances per request
ITEMS_PER_REQUEST = 5

# Loop through each generated dataframe, convert
for i, df in tqdm(
    enumerate(
        query_bigquery_chunks(
            max_rows=BQ_NUM_ROWS, rows_per_chunk=BQ_CHUNK_SIZE, start_chunk=START_CHUNK
        )
    ),
    total=BQ_NUM_CHUNKS - START_CHUNK,
    position=-1,
    desc="Chunk of rows from BigQuery",
):
    # Create a unique output file for each chunk
    chunk_path = embeddings_file_path.joinpath(
        f"{embeddings_file_path.stem}_{i+START_CHUNK}.json"
    )
    with open(chunk_path, "a") as f:
        id_chunk = df.id

        # Convert batch to embeddings
        is_successful, question_chunk_embeddings = encode_text_to_embedding_batched(
            sentences=df.title_with_body.to_list(),
            api_calls_per_second=API_CALLS_PER_SECOND,
            batch_size=ITEMS_PER_REQUEST,
        )

        # Append to file
        embeddings_formatted = [
            json.dumps(
                {
                    "id": str(id),
                    "embedding": [str(value) for value in embedding],
                }
            )
            + "\n"
            for id, embedding in zip(id_chunk[is_successful], question_chunk_embeddings)
        ]
        f.writelines(embeddings_formatted)

        # Delete the DataFrame and any other large data structures
        del df
        gc.collect()

Chunk of rows from BigQuery:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

### Task 6. Upload embeddings to Cloud Storage

Upload the text-embeddings to Cloud Storage, so that Vertex AI Vector Search can access them later.

In [17]:
BUCKET_URI = f"gs://{PROJECT_ID}-unique"

! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

remote_folder = f"{BUCKET_URI}/{embeddings_file_path.stem}/"
! gsutil -m cp -r {embeddings_file_path}/* {remote_folder}

Creating gs://qwiklabs-gcp-03-e0549e42556f-unique/...
Copying file:///var/tmp/tmp64_knyge/tmp64_knyge_0.json [Content-Type=application/json]...
Copying file:///var/tmp/tmp64_knyge/tmp64_knyge_3.json [Content-Type=application/json]...
Copying file:///var/tmp/tmp64_knyge/tmp64_knyge_2.json [Content-Type=application/json]...
Copying file:///var/tmp/tmp64_knyge/tmp64_knyge_1.json [Content-Type=application/json]...
Copying file:///var/tmp/tmp64_knyge/tmp64_knyge_4.json [Content-Type=application/json]...
- [5/5 files][ 12.3 MiB/ 12.3 MiB] 100% Done                                    
Operation completed over 5 objects/12.3 MiB.                                     


### Task 7. Create an Index in Vertex AI Vector Search for your embeddings

In [18]:
DISPLAY_NAME = "stack_overflow"
DESCRIPTION = "question titles and bodies from stackoverflow"

Create the index. Notice that the index reads the embeddings from the Cloud Storage bucket. The indexing process can take from 45 minutes up to 60 minutes. Wait for completion, and then proceed. You can open a different Google Cloud Console page, navigate to Vertex AI Vector search, and see how the index is being created.

In [19]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

DIMENSIONS = 768

tree_ah_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=DISPLAY_NAME,
    contents_delta_uri=remote_folder,
    dimensions=DIMENSIONS,
    approximate_neighbors_count=150,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    leaf_node_embedding_count=500,
    leaf_nodes_to_search_percent=80,
    description=DESCRIPTION,
)

Creating MatchingEngineIndex
Create MatchingEngineIndex backing LRO: projects/209209658799/locations/us-east4/indexes/7816639268092116992/operations/7942094094577172480
MatchingEngineIndex created. Resource name: projects/209209658799/locations/us-east4/indexes/7816639268092116992
To use this MatchingEngineIndex in another session:
index = aiplatform.MatchingEngineIndex('projects/209209658799/locations/us-east4/indexes/7816639268092116992')


In [20]:
INDEX_RESOURCE_NAME = tree_ah_index.resource_name
INDEX_RESOURCE_NAME

'projects/209209658799/locations/us-east4/indexes/7816639268092116992'

In [21]:
## Using the resource name, you can retrieve an existing MatchingEngineIndex.
tree_ah_index = aiplatform.MatchingEngineIndex(index_name=INDEX_RESOURCE_NAME)

In [22]:
## Create an IndexEndpoint so that it can be accessed via an API.
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=DISPLAY_NAME,
    description=DISPLAY_NAME,
    public_endpoint_enabled=True,
)

Creating MatchingEngineIndexEndpoint
Create MatchingEngineIndexEndpoint backing LRO: projects/209209658799/locations/us-east4/indexEndpoints/3679344138538450944/operations/1578507821102661632
MatchingEngineIndexEndpoint created. Resource name: projects/209209658799/locations/us-east4/indexEndpoints/3679344138538450944
To use this MatchingEngineIndexEndpoint in another session:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/209209658799/locations/us-east4/indexEndpoints/3679344138538450944')


In [23]:
## Deploy your index to the created endpoint. This can take up to 15 minutes.
DEPLOYED_INDEX_ID = "deployed_index_id_unique"
DEPLOYED_INDEX_ID

my_index_endpoint = my_index_endpoint.deploy_index(
    index=tree_ah_index, deployed_index_id=DEPLOYED_INDEX_ID
)

my_index_endpoint.deployed_indexes

Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/209209658799/locations/us-east4/indexEndpoints/3679344138538450944
Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/209209658799/locations/us-east4/indexEndpoints/3679344138538450944/operations/2661623531485265920
MatchingEngineIndexEndpoint index_endpoint Deployed index. Resource name: projects/209209658799/locations/us-east4/indexEndpoints/3679344138538450944


[id: "deployed_index_id_unique"
index: "projects/209209658799/locations/us-east4/indexes/7816639268092116992"
create_time {
  seconds: 1737066060
  nanos: 767854000
}
index_sync_time {
  seconds: 1737067639
  nanos: 999489000
}
automatic_resources {
  min_replica_count: 2
  max_replica_count: 2
}
deployment_group: "default"
]

Verify number of declared items matches the number of embeddings. Each IndexEndpoint can have multiple indexes deployed to it. For each index, you can retrieve the number of deployed vectors using the `index_endpoint._gca_resource.index_stats.vectors_count`. The numbers may not match exactly due to potential rate-limiting failures incurred when using the embedding service.

In [24]:
number_of_vectors = sum(
    aiplatform.MatchingEngineIndex(
        deployed_index.index
    )._gca_resource.index_stats.vectors_count
    for deployed_index in my_index_endpoint.deployed_indexes
)

print(f"Expected: {BQ_NUM_ROWS}, Actual: {number_of_vectors}")

Expected: 5000, Actual: 570


### Task 8. Create online queries

After you build your indexes, you may query against the deployed index to find nearest neighbors.

**Note**: For the `DOT_PRODUCT_DISTANCE` distance type, the "distance" property returned with each MatchNeighbor actually refers to the similarity.

In [25]:
test_embeddings = encode_texts_to_embeddings(sentences=["Install GPU for Tensorflow"])

NUM_NEIGHBOURS = 10

response = my_index_endpoint.find_neighbors(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=test_embeddings,
    num_neighbors=NUM_NEIGHBOURS,
)

response

[[MatchNeighbor(id='61257444', distance=0.469961017370224, sparse_distance=None, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[], sparse_embedding_values=[], sparse_embedding_dimensions=[]),
  MatchNeighbor(id='62798017', distance=0.46461033821105957, sparse_distance=None, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[], sparse_embedding_values=[], sparse_embedding_dimensions=[]),
  MatchNeighbor(id='56910099', distance=0.45542389154434204, sparse_distance=None, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[], sparse_embedding_values=[], sparse_embedding_dimensions=[]),
  MatchNeighbor(id='22264626', distance=0.45417413115501404, sparse_distance=None, feature_vector=[], crowding_tag='0', restricts=[], numeric_restricts=[], sparse_embedding_values=[], sparse_embedding_dimensions=[]),
  MatchNeighbor(id='46385086', distance=0.4423808157444, sparse_distance=None, feature_vector=[], crowding_tag='0', restricts=[], nume

In [26]:
## Verify that the retrieved results are relevant by checking the StackOverflow links.
for match_index, neighbor in enumerate(response[0]):
    print(f"https://stackoverflow.com/questions/{neighbor.id}")

https://stackoverflow.com/questions/61257444
https://stackoverflow.com/questions/62798017
https://stackoverflow.com/questions/56910099
https://stackoverflow.com/questions/22264626
https://stackoverflow.com/questions/46385086
https://stackoverflow.com/questions/22301307
https://stackoverflow.com/questions/61230538
https://stackoverflow.com/questions/44416585
https://stackoverflow.com/questions/67171256
https://stackoverflow.com/questions/44331812


### Task 9. Clean up the Google Cloud environment

To clean up all Google Cloud resources used in this project, you can delete the Google Cloud project you used for the lab. You can also manually delete resources that you created by running the following code.

In [ ]:
import os

delete_bucket = False

# Force undeployment of indexes and delete endpoint
my_index_endpoint.delete(force=True)

# Delete indexes
tree_ah_index.delete()

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}